In [2]:
import time
import threading
import warnings
from datetime import datetime
from typing import Dict , Optional
import pandas as pd
warnings.filterwarnings('ignore')

# interactive broker API live

In [ ]:
# !pip install ibapi
# !pip install ib_insync

In [5]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.common import BarData

In [9]:
class TradingApp(EClient,EWrapper):
    def __init__(self)->None:
        EClient.__init__(self,self)
        self.data: Dict[int,pd.DataFrame]={}
    def error(self,reqId:int, erroCode:int,errorString:str, advanced:any)->None:
        print(f'Error: ',{reqId},{erroCode},{errorString})
    def get_historical_data(self,reqId:int,contract:Contract)->pd.DataFrame:
        self.data[reqId]=pd.DataFrame(columns=['time','high','low','close'])
        self.data[reqId].set_index('time',inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(4)
        return self.data[reqId]
    def historicalData(self, reqId:int, bar:BarData)->None:
        df=self.data[reqId]
        df.loc[pd.to_datetime(bar.date,unit='s'),['high','low','close']]=[bar.high,bar.low,bar.close]
        df=df.astype(float)
        self.data[reqId]=df
    @staticmethod
    def get_stock_contract(symbol:str)->Contract:
        contract=Contract()
        contract.symbol=symbol
        contract.secType='STK'
        contract.exchange='USD'
        return contract
        
    
        


# you need to have TWS API and Trader WorkStation